In [81]:
import pandas as pd
import plotly.express as px

# Load dataset
df = pd.read_csv("PSDP_2024_25_verified.csv")
df.columns


Index(['Sr#', 'Project_Title', 'Approval_Status', 'Approved_Date',
       'Total_Cost', 'Foreign Loan', 'Expenditure_2024', 'Throwforward_2025',
       'Rupee_Funding', 'Allocation_2024_25', 'Type', 'Ministry',
       'Utilization%', 'Approval_Year', 'Approval_Month'],
      dtype='object')

In [82]:
# chart_01_budget_utilization_by_ministry.py
import pandas as pd
import plotly.express as px
import os

# Load verified PSDP dataset
df = pd.read_csv("PSDP_2024_25_verified.csv")

# ---- Compute Utilization ----
df["Budget_Utilization_%"] = (df["Expenditure_2024"] / df["Allocation_2024_25"]) * 100
df["Budget_Utilization_%"] = df["Budget_Utilization_%"].clip(0, 200)  # safety cap

# ---- Clean up ----
df = df.dropna(subset=["Ministry", "Budget_Utilization_%"])
ministry_util = (
    df.groupby("Ministry", as_index=False)
    .agg({"Budget_Utilization_%": "mean", "Allocation_2024_25": "sum"})
    .sort_values("Budget_Utilization_%", ascending=False)
)

# ---- Plot ----
fig = px.bar(
    ministry_util,
    x="Budget_Utilization_%",
    y="Ministry",
    orientation="h",
    color="Budget_Utilization_%",
    color_continuous_scale="Viridis",
    text=round(ministry_util["Budget_Utilization_%"], 1),
    title="💰 Average Budget Utilization (%) by Ministry — PSDP 2024-25"
)
fig.update_traces(textposition="outside")
fig.update_layout(
    xaxis_title="Average Utilization (%)",
    yaxis_title="Ministry",
    title_font=dict(size=20, family="Arial Bold"),
    plot_bgcolor="white",
    height=650,
    margin=dict(l=120, r=40, t=80, b=40)
)

# # ---- Save and Show ----
# os.makedirs("charts", exist_ok=True)
# fig.write_html("charts/chart_01_budget_utilization_by_ministry.html")
fig.show()


In [83]:
import math

# --- Calculate dynamic height based on ministries ---
bar_count = len(ministry_util)
fig_height = max(500, bar_count * 28)  # 28 px per bar for breathing space

# --- Create improved bar chart ---
fig = px.bar(
    ministry_util,
    x="Budget_Utilization_%",
    y="Ministry",
    orientation="h",
    color="Budget_Utilization_%",
    color_continuous_scale="Viridis",
    text=round(ministry_util["Budget_Utilization_%"], 1),
)

# --- Update layout for clarity ---
fig.update_traces(
    textposition="outside",
    marker_line_width=0.5,
    marker_line_color="white",
)

fig.update_layout(
    title={
        "text": "💰 <b>Average Budget Utilization (%) by Ministry — PSDP 2024–25</b>",
        "x": 0.5,   # Center align title
        "xanchor": "center",
        "yanchor": "top"
    },
    xaxis_title="Average Utilization (%)",
    yaxis_title="Ministry",
    title_font=dict(size=22, family="Arial Bold"),
    font=dict(size=11, family="Arial"),
    plot_bgcolor="white",
    paper_bgcolor="white",
    height=fig_height,
    margin=dict(l=160, r=40, t=80, b=40),
    coloraxis_colorbar=dict(title="Utilization (%)")
)


fig.write_html("charts/chart_01_budget_utilization_by_ministry.html")
fig.show()


In [ ]:
df['Type'].unique()

array(['On-going', 'New', 'Ministry of Railways (3rd Revised)',
       'Ministry of Railways, Islamabad', 'Ministry of Law and Justice',
       'Ministry of Law and Justice and', 'Ministry of Railways (2nd'],
      dtype=object)

* There should be only two types in type column so now we are replacing it with going and on going only.

In [ ]:
# --- Clean and standardize the Type column ---
df["Type"] = df["Type"].astype(str).str.strip().str.lower()

# Replace similar words and messy text
df["Type"] = df["Type"].replace({
    "on-going": "Ongoing",
    "ongoing": "Ongoing",
    "new": "New",
    "nan": "Ongoing"
})

# Convert anything else (like ministry names, revisions) into 'Ongoing'
df.loc[~df["Type"].isin(["Ongoing", "New"]), "Type"] = "Ongoing"


In [ ]:
df['Type'].unique()

array(['Ongoing', 'New'], dtype=object)

In [ ]:
# chart_03_utilization_by_type_fixed.py
import pandas as pd
import plotly.express as px
import os


# --- Compute Utilization % ---
df["Budget_Utilization_%"] = (df["Expenditure_2024"] / df["Allocation_2024_25"]) * 100
df["Budget_Utilization_%"] = df["Budget_Utilization_%"].clip(0, 200)

# Drop invalid values
df = df.dropna(subset=["Type", "Budget_Utilization_%"])

# --- Group by clean project type ---
type_util = (
    df.groupby("Type", as_index=False)
    .agg({"Budget_Utilization_%": "mean", "Total_Cost": "sum"})
    .sort_values("Budget_Utilization_%", ascending=False)
)

# --- Plot clean two-bar chart ---
fig = px.bar(
    type_util,
    x="Type",
    y="Budget_Utilization_%",
    color="Type",
    text=round(type_util["Budget_Utilization_%"], 1),
    color_discrete_sequence=["#1F77B4", "#2ECC71"],  # blue for new, green for ongoing
    title="📊 Average Budget Utilization by Project Type — PSDP 2024-25"
)

fig.update_traces(textposition="outside")

fig.update_layout(
    title_x=0.5,
    title_font=dict(size=22, family="Arial Bold"),
    xaxis_title="Project Type",
    yaxis_title="Average Utilization (%)",
    plot_bgcolor="white",
    paper_bgcolor="white",
    font=dict(size=12, family="Arial"),
    height=550,
    margin=dict(l=60, r=40, t=80, b=60),
)

# Save and show
# os.makedirs("charts", exist_ok=True)
fig.write_html("charts/chart_03_utilization_by_type_fixed.html")
fig.show()


In [ ]:
import pandas as pd
import plotly.express as px
import os

# Load dataset
df = pd.read_csv("PSDP_2024_25_verified.csv")

# ---- Calculate Funding Gap ----
df["Funding_Gap"] = df["Total_Cost"] - df["Allocation_2024_25"]
df["Funding_Gap"] = df["Funding_Gap"].clip(lower=0)  # avoid negatives

# ---- Group by Ministry ----
ministry_gap = (
    df.groupby("Ministry", as_index=False)
    .agg({"Total_Cost": "sum", "Allocation_2024_25": "sum", "Funding_Gap": "mean"})
    .sort_values("Funding_Gap", ascending=False)
)

# ---- Plot ----
fig = px.bar(
    ministry_gap.head(15),
    x="Funding_Gap",
    y="Ministry",
    orientation="h",
    color="Funding_Gap",
    color_continuous_scale="Reds",
    text=round(ministry_gap.head(15)["Funding_Gap"], 1),
    title="💸 Funding Gap (Total Cost vs Annual Allocation) — PSDP 2024–25",
)

fig.update_layout(
    title_x=0.5,
    title_font=dict(size=22, family="Arial Bold"),
    xaxis_title="Average Funding Gap (Rs Million)",
    yaxis_title="Ministry",
    plot_bgcolor="white",
    paper_bgcolor="white",
    font=dict(size=12, family="Arial"),
    height=650,
    margin=dict(l=180, r=40, t=80, b=60),
)

# # ---- Save and Show ----
os.makedirs("charts", exist_ok=True)
fig.write_html("charts/chart_04_funding_gap_by_ministry.html")
fig.show()


In [ ]:
import pandas as pd
import plotly.express as px
import os

# Load dataset
df = pd.read_csv("PSDP_2024_25_verified.csv")

# ---- Calculate Throwforward Ratio ----
df["Throwforward_Ratio"] = (df["Throwforward_2025"] / df["Allocation_2024_25"]) * 100
df["Throwforward_Ratio"] = df["Throwforward_Ratio"].clip(0, 300)  # limit extremes

# ---- Clean missing values ----
df = df.dropna(subset=["Ministry", "Throwforward_Ratio"])

# ---- Group by Ministry ----
ministry_forward = (
    df.groupby("Ministry", as_index=False)
    .agg({"Throwforward_Ratio": "mean", "Throwforward_2025": "sum", "Allocation_2024_25": "sum"})
    .sort_values("Throwforward_Ratio", ascending=False)
)

# ---- Plot ----
fig = px.bar(
    ministry_forward.head(15),
    x="Throwforward_Ratio",
    y="Ministry",
    orientation="h",
    color="Throwforward_Ratio",
    color_continuous_scale="Tealgrn",
    text=round(ministry_forward.head(15)["Throwforward_Ratio"], 1),
    title="📈 Throwforward 2025 vs Current Allocation — Future Funding Pressure by Ministry",
)

# ---- Styling ----
fig.update_traces(textposition="outside")
fig.update_layout(
    title_x=0.5,
    title_font=dict(size=22, family="Arial Bold"),
    xaxis_title="Throwforward Ratio (%)",
    yaxis_title="Ministry",
    plot_bgcolor="white",
    paper_bgcolor="white",
    font=dict(size=12, family="Arial"),
    height=650,
    margin=dict(l=180, r=40, t=80, b=60),
)

# # ---- Save & Show ----
os.makedirs("charts", exist_ok=True)
fig.write_html("charts/chart_05_throwforward_vs_allocation.html")
fig.show()
